<a href="https://colab.research.google.com/github/leohcar/P_IA025A_2022S1/blob/main/segunda_entrega/Proyecto_encoder_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Encoder

In [ ]:
nome = 'Carlos Leonardo Ancasi Hinostroza'

print(f'Meu nome é {nome}')

Meu nome é Carlos Leonardo Ancasi Hinostroza


# Importação das bibliotecas

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torchvision
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from torchvision.datasets import MNIST

In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed Jul  6 15:38:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
print('GPU available:', device)

GPU available: cuda


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

#Dataset e dataloader

In [ ]:
!mkdir ./data

# transform = torchvision.transforms.Compose(
#     [torchvision.transforms.Resize([224,224]),
#      torchvision.transforms.ToTensor(),
#      torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
transform = torchvision.transforms.Compose(
     [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
print(f'numero de imagenes {len(train_dataset)}')
img, target = train_dataset[0]
print(img.size())

numero de imagenes 50000
torch.Size([3, 32, 32])


In [ ]:
train_size = 40000
val_size = 10000
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 800
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([50, 3, 32, 32])
Valores mínimo e máximo dos pixels:  tensor(-1.) tensor(1.)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


#Bloco Básico da Resnet

In [ ]:
from collections import OrderedDict
from torch import nn
from torch import Tensor


class MyBlock(nn.Module):

    def __init__(
        self,
        inplanes: int,
        planes: int,
        identity_downsample=None,
        stride: int = 1,
    ) -> None:
        super(MyBlock, self).__init__()
        # Escreva seu código aqui.
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        # Escreva seu código aqui.
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.identity_downsample is not None:
            identity = self.identity_downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [ ]:
class CNN_resnet(nn.Module):
    def __init__(self, image_channels, num_classes):

        super(CNN_resnet,self).__init__()

        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2)
        self.layer4 = self.__make_layer(256, 512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        self.fc = nn.Linear(512, num_classes)
      
    def __make_layer(self, in_channels, out_channels, stride):

        identity_downsample = None
        
        if stride != 1:
            identity_downsample = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0,bias=False),
                nn.BatchNorm2d(out_channels))
        return nn.Sequential(MyBlock(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride),
            MyBlock(out_channels, out_channels),
            nn.Dropout(0.3))
    
    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avgpool(out)
        out = out.view(x.shape[0], -1)
        out = self.fc(out)
        return out


In [ ]:
model = CNN_resnet(3,100).to(device)

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 11227876


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_save_name = 'model_resnet18_CIFAR_01.pt'
path_modelo = F"/content/drive/My Drive/modelo1/{model_save_name}"

In [ ]:
# Iniciar o treinamento
from tqdm import tqdm

n_epochs = 20
lr = 3e-4

img_channels = 3
num_classes = 100

model = CNN_resnet(img_channels,num_classes).to(device)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

checkpoint = {
    'img_channels': img_channels,
    'num_classes': num_classes,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'epochs': [],
    'loss_train': [],
    'loss_val': [],
    'accuracy_history': [],
    }
torch.save(checkpoint, path_modelo)

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = []
accuracy_history = []
total_trained_samples = 0

with torch.no_grad():
    loss_accuracy = [list(validation_step(val_input_ids.to(device), val_target_ids.to(device)))
        for val_input_ids, val_target_ids in val_dataloader]
    
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()
    print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
    accuracy_history.append(mean_accuracy.item())

    checkpoint['accuracy_history'].append(mean_accuracy.item())
    checkpoint['loss_val'].append(mean_loss.item())
    torch.save(checkpoint, path_modelo)
    
for i in range(n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())

    torch.save(checkpoint, path_modelo)


Accuracy: 0.0054 Loss_Val: 4.605927608013153


Training: 100%|██████████| 800/800 [00:24<00:00, 32.59it/s]


Epoch: 0/19 Loss_Train: 3.7273900508880615
Accuracy: 0.139 Loss_Val: 3.6200682306289673


Training: 100%|██████████| 800/800 [00:24<00:00, 32.73it/s]


Epoch: 1/19 Loss_Train: 3.321450710296631
Accuracy: 0.20910000000000004 Loss_Val: 3.175502732992172


...

Epoch: 17/19 Loss_Train: 1.5310258865356445
Accuracy: 0.4593 Loss_Val: 2.104803954362869


Training: 100%|██████████| 800/800 [00:25<00:00, 31.62it/s]


Epoch: 18/19 Loss_Train: 1.3641448020935059
Accuracy: 0.45890000000000003 Loss_Val: 2.147762584686279


Training: 100%|██████████| 800/800 [00:25<00:00, 31.49it/s]


Epoch: 19/19 Loss_Train: 1.4664912223815918
Accuracy: 0.46540000000000004 Loss_Val: 2.1404479509592056


In [ ]:
# continuar com o  treino
from tqdm import tqdm

n_epochs = 60
lr = 3e-4
path_modelo_load='model_resnet18_CIFAR_02.pt'
path_modelo_save='model_resnet18_CIFAR_03.pt'
checkpoint = torch.load(path_modelo_load)

model = CNN_resnet(checkpoint['img_channels'],checkpoint['num_classes']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)
    loss.backward()
    optimizer.step()

    return loss.item()

def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids)

    _, y_pred = torch.max(logits,1)
    accuracy_item = (y_pred.cpu().numpy() == target_ids.cpu().numpy()).mean()

    return accuracy_item,loss.item()

epochs = []
loss_history = []
loss_epoch_end = checkpoint['loss_train']
accuracy_history = checkpoint['accuracy_history']
total_trained_samples = 0
epochs_init = checkpoint['epochs'][-1]+1

for s in zip(checkpoint['accuracy_history'],checkpoint['loss_val'],checkpoint['epochs'],checkpoint['loss_train']):
    print(f'Accuracy: {s[0]} Loss_Val: {s[1]}')
    print(f'')
    print(f'Epoch: {s[2]:d}/{n_epochs - 1:d} Loss_Train: {s[3]}')
print(f"Accuracy: { checkpoint['accuracy_history'][-1] } Loss_Val: {checkpoint['loss_val'][-1]}")

    
for i in range(epochs_init,n_epochs):
    for x_train, y_train in tqdm(train_dataloader, total=len(train_dataloader), desc='Training'):
        loss = train_step(x_train.to(device), y_train.to(device))
        
        total_trained_samples += x_train.size(0)
        epochs.append(total_trained_samples / len(train_dataset))
        loss_history.append(loss)
    
    loss_epoch_end.append(loss)
    print(f'Epoch: {i:d}/{n_epochs - 1:d} Loss_Train: {loss}')
    checkpoint['epochs'].append(i)
    checkpoint['loss_train'].append(loss)
    
    with torch.no_grad():
        loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
            for val_input_ids, val_target_ids in val_dataloader]
        
        mean_accuracy = np.array(loss_accuracy)[:,0].mean()
        mean_loss = np.array(loss_accuracy)[:,1].mean()
        print(f'Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')
        accuracy_history.append(mean_accuracy.item())

        checkpoint['accuracy_history'].append(mean_accuracy.item())
        checkpoint['loss_val'].append(mean_loss.item())

    torch.save(checkpoint, path_modelo_save)


Accuracy: 0.0054 Loss_Val: 4.605927608013153

Epoch: 0/59 Loss_Train: 3.7273900508880615
Accuracy: 0.139 Loss_Val: 3.6200682306289673

Epoch: 1/59 Loss_Train: 3.321450710296631
Accuracy: 0.20910000000000004 Loss_Val: 3.175502732992172

Epoch: 2/59 Loss_Train: 3.0338246822357178
Accuracy: 0.24880000000000002 Loss_Val: 2.9630845844745637

Epoch: 3/59 Loss_Train: 2.6470770835876465
Accuracy: 0.29669999999999996 Loss_Val: 2.7318737614154815

Epoch: 4/59 Loss_Train: 2.515414237976074
Accuracy: 0.32280000000000003 Loss_Val: 2.616809132695198

Epoch: 5/59 Loss_Train: 2.321904420852661
Accuracy: 0.3539 Loss_Val: 2.4696280962228774

Epoch: 6/59 Loss_Train: 2.7074673175811768
Accuracy: 0.3726 Loss_Val: 2.3994606828689573

Epoch: 7/59 Loss_Train: 2.256422519683838
Accuracy: 0.3807 Loss_Val: 2.3772034394741057

Epoch: 8/59 Loss_Train: 2.148145914077759
Accuracy: 0.3917 Loss_Val: 2.3069273120164873

Epoch: 9/59 Loss_Train: 2.1474103927612305
Accuracy: 0.40739999999999993 Loss_Val: 2.252385718822479

Training: 100%|██████████| 800/800 [00:24<00:00, 32.15it/s]


Epoch: 40/59 Loss_Train: 1.4808993339538574
Accuracy: 0.4686 Loss_Val: 2.127242791056633


Training: 100%|██████████| 800/800 [00:24<00:00, 32.56it/s]


Epoch: 41/59 Loss_Train: 1.2375434637069702
Accuracy: 0.47240000000000004 Loss_Val: 2.1607748943567278


...

Training: 100%|██████████| 800/800 [00:24<00:00, 33.12it/s]


Epoch: 56/59 Loss_Train: 0.6273702383041382
Accuracy: 0.47450000000000003 Loss_Val: 2.604168074131012


Training: 100%|██████████| 800/800 [00:24<00:00, 32.89it/s]


Epoch: 57/59 Loss_Train: 0.6101028323173523
Accuracy: 0.48190000000000005 Loss_Val: 2.5528094500303267


Training:  83%|████████▎ | 667/800 [00:20<00:04, 32.65it/s]

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=batch_size)

with torch.no_grad():
    loss_accuracy = [validation_step(val_input_ids.to(device), val_target_ids.to(device))
        for val_input_ids, val_target_ids in val_dataloader]
        
    mean_accuracy = np.array(loss_accuracy)[:,0].mean()
    mean_loss = np.array(loss_accuracy)[:,1].mean()

print(f'TEST: Accuracy: {mean_accuracy} Loss_Val: {mean_loss}')

TEST: Accuracy: 0.4772 Loss_Val: 2.6944082736968995
